In [ ]:
%load_ext Cython

In [ ]:
%%cython
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
import numpy as np
cimport numpy as cnp
from numpy import sin, cos, tan, arctan, arctan2, arcsin, degrees, radians
cnp.import_array()
cimport fastspa._core as fastspa


# cimport fastspa._lib as lib
cdef int TOPOCENTRIC_RIGHT_ASCENSION = 0
cdef int TOPOCENTRIC_DECLINATION = 1
cdef int APARENT_SIDEREAL_TIME = 2
cdef int EQUATOIRAL_HORIZONAL_PARALAX = 3


ctypedef cnp.float64_t DTYPE_t 
DTYPE = np.float64

# DTYPE = cnp.float64
# =============================================================================
# @cython.boundscheck(False)
# @cython.wraparound(False)
cdef cnp.ndarray[DTYPE_t, ndim=3]  topocentric_parallax_right_ascension_and_declination(
    (int, int) shape,
    cnp.ndarray[DTYPE_t, ndim=1] delta,   # δ geocentric sun declination
    cnp.ndarray[DTYPE_t, ndim=1] H,       # H local hour angle
    cnp.ndarray[DTYPE_t, ndim=1] E,       # E observer elevation
    cnp.ndarray[DTYPE_t, ndim=1] lat,     # observer latitude
    cnp.ndarray[DTYPE_t, ndim=1] xi,      # ξ equatorial horizontal parallax
): # type: ignore
    # cdef double u, x, y, Phi, delta_alpha, delta_p
    cdef cnp.ndarray[DTYPE_t, ndim=3] out 
    out = np.empty((2,) + shape, dtype=DTYPE)
    delta = radians(delta)          # δ
    xi = radians(xi)                # ξ
    Phi = radians(lat)              # ϕ

    # - 3.12.2. Calculate the term u (in radians)
    u = (
        arctan(0.99664719 * tan(Phi))
    ) # u = Acrtan(0.99664719 * tan ϕ)

    # - 3.12.3. Calculate the term x,
    x = (
        cos(u) + E / 6378140 * cos(Phi)
    ) # x = cosu + E / 6378140 * cos ϕ

    # - 3.12.4. Calculate the term y
    y = (
        0.99664719 * sin(u) + E / 6378140 * sin(Phi)
    ) # y = 0.99664719 * sin u + E / 6378140 * sin ϕ

    # - 3.12.5. Calculate the parallax in the sun right ascension (in degrees),
    delta_alpha = arctan2(
        -x * sin(xi) * sin(H),
        cos(delta) - x * sin(xi) * cos(H)
    ) # ∆α = Arctan2(-x *sin ξ *sin H / cosδ − x * sin ξ * cos H)

    delta_p = arcsin(
        sin(delta) - y * sin(xi) * cos(delta_alpha)
    ) # ∆' = Arcsin(sinδ − y * sin ξ * cos ∆α)
    # return degrees(delta_alpha), degrees(delta_p)
    out[0] = degrees(delta_alpha)
    out[1] = degrees(delta_p)
    return out
# =============================================================================
cdef cnp.ndarray main_old():
    cdef cnp.ndarray[DTYPE_t, ndim=1] lat, lon, elev
    lats = np.linspace(25, 50, 100).astype(DTYPE)[np.newaxis]
    lons = np.linspace(-125, -65, 100).astype(DTYPE)[np.newaxis]
    elev = np.zeros_like(lats)



    # cdef cnp.ndarray[DTYPE_t, ndim=1] H, H_p, delta_alpha, delta_p, e0, e, theta, theta0, gamma, phi,xi
    time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')

    shape = (time.shape[0], lats.shape[1])
    tc = np.asfarray(fastspa.time_components(time))[:, :, np.newaxis]
    v       = tc[APARENT_SIDEREAL_TIME]                                  # ν
    xi      = tc[EQUATOIRAL_HORIZONAL_PARALAX]                           # ξ
    delta   = tc[TOPOCENTRIC_DECLINATION]                                # δ
    alpha   = tc[TOPOCENTRIC_RIGHT_ASCENSION]
    H = (v + lons - alpha) % 360       
    

    cdef cnp.ndarray[DTYPE_t, ndim=3] result
    result = topocentric_parallax_right_ascension_and_declination(
        # '\n',
        shape, 
        delta,
        H, 
        elev, 
        lats, xi
    )
    return result




In [ ]:
%%cython
cimport cython
import cython

ctypedef fused Bool:
    # False
    bint
    int

In [ ]:
%%cython
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
import numpy as np
cimport numpy as cnp
from numpy import sin, cos, tan, arctan, arctan2, arcsin, degrees, radians
from numpy.typing import ArrayLike
cnp.import_array()
cimport fastspa._core as fastspa


# cimport fastspa._lib as lib
cdef int TOPOCENTRIC_RIGHT_ASCENSION = 0
cdef int TOPOCENTRIC_DECLINATION = 1
cdef int APARENT_SIDEREAL_TIME = 2
cdef int EQUATOIRAL_HORIZONAL_PARALAX = 3


ctypedef cnp.float64_t DTYPE_t 
DTYPE = np.float64



cdef  topocentric_parallax_right_ascension_and_declination(
    double delta,   # δ geocentric sun declination
    cnp.ndarray[DTYPE_t, ndim=1] H,       # H local hour angle
    cnp.ndarray[DTYPE_t, ndim=1] E,       # E observer elevation
    cnp.ndarray[DTYPE_t, ndim=1] lat,     # observer latitude
    double xi,      # ξ equatorial horizontal parallax
) : # type: ignore
    cdef cnp.ndarray[DTYPE_t, ndim=1] u, x, y, Phi, delta_alpha, delta_p
    
    delta = radians(delta)          # δ
    xi = radians(xi)                # ξ
    Phi = radians(lat)              # ϕ

    # - 3.12.2. Calculate the term u (in radians)
    u = (
        arctan(0.99664719 * tan(Phi))
    ) # u = Acrtan(0.99664719 * tan ϕ)

    # - 3.12.3. Calculate the term x,
    x = (
        cos(u) + E / 6378140 * cos(Phi)
    ) # x = cosu + E / 6378140 * cos ϕ

    # - 3.12.4. Calculate the term y
    y = (
        0.99664719 * sin(u) + E / 6378140 * sin(Phi)
    ) # y = 0.99664719 * sin u + E / 6378140 * sin ϕ

    # - 3.12.5. Calculate the parallax in the sun right ascension (in degrees),
    delta_alpha = arctan2(
        -x * sin(xi) * sin(H),
        cos(delta) - x * sin(xi) * cos(H)
    ) # ∆α = Arctan2(-x *sin ξ *sin H / cosδ − x * sin ξ * cos H)

    delta_p = arcsin(
        sin(delta) - y * sin(xi) * cos(delta_alpha)
    ) # ∆' = Arcsin(sinδ − y * sin ξ * cos ∆α)


    return degrees(delta_alpha), degrees(delta_p)

cdef topocentric_azimuth_angle(
    cnp.ndarray[DTYPE_t, ndim=1] phi,         # φ observer latitude
    cnp.ndarray[DTYPE_t, ndim=1] delta_p,     # δ’ topocentric sun declination
    cnp.ndarray[DTYPE_t, ndim=1] H_p,         # H’ topocentric local hour angle
    double P,           # P is the annual average local pressure (in millibars)
    double T,           # T is the annual average local temperature (in degrees Celsius)
    double refraction
): # type: ignore
    cdef cnp.ndarray[DTYPE_t, ndim=1] e, e0, delta_e, theta, theta0
    # - in radians
    phi = radians(phi)
    delta_p = radians(delta_p)
    H_p = radians(H_p)

    # 3.14.1
    e0 = (
        arcsin(sin(phi)* sin(delta_p) + cos(phi) * cos(delta_p) * cos(H_p))
    ) # e0 = Arcsin(sin ϕ *sin δ'+ cos ϕ *cos δ'*cos H') 

    # - in degrees
    e0 = degrees(e0)

    # 3.14.2
    delta_e = (
        (P / 1010.0) * (283.0 / (273 + T))  * 1.02 / (60 * tan(radians(e0 + 10.3 / (e0 + 5.11))))
    ) # ∆e = (P / 1010) * (283 / (273 + T)) * 1.02 / (60 * tan(radians(e0 + 10.3 / (e0 + 5.11))))
    # Note that ∆e = 0 when the sun is below the horizon.
    delta_e *= e0 >= -1.0 * (0.26667 + refraction)

    # 3.14.3. Calculate the topocentric elevation angle, e (in degrees), 
    e =  e0 + delta_e       # e = e0 + ∆e

    # 3.14.4. Calculate the topocentric zenith angle, 2 (in degrees),
    theta = 90 - e          # θ = 90 − e
    theta0 = 90 - e0        # θ0 = 90 − e0

    return e, e0, theta, theta0 

cdef topocentric_astronomers_azimuth(
    cnp.ndarray[DTYPE_t, ndim=1] topocentric_local_hour_angle,
    cnp.ndarray[DTYPE_t, ndim=1] topocentric_sun_declination,
    cnp.ndarray[DTYPE_t, ndim=1] observer_latitude
): # type: ignore
    # cdef double topocentric_local_hour_angle_rad, phi, gamma
    topocentric_local_hour_angle_rad = radians(topocentric_local_hour_angle)
    phi = radians(observer_latitude)
    gamma = arctan2(
        sin(topocentric_local_hour_angle_rad), 
        cos(topocentric_local_hour_angle_rad) * sin(phi) - tan(radians(topocentric_sun_declination)) * cos(phi)
    )
    
    return degrees(gamma) % 360

cdef c_main(
    # int num_t,
    double[:, :] tc,
    cnp.ndarray[DTYPE_t, ndim=1] lat,
    cnp.ndarray[DTYPE_t, ndim=1] lon,
    cnp.ndarray[DTYPE_t, ndim=1] E,
    double P,
    double T,
    double refct,
    int num_threads
):
    cdef int i, n
    cdef double v, xi, delta, alpha
    # cdef cnp.ndarray[DTYPE_t, ndim=1] H, H_p, delta_alpha, delta_p#, e0#, e
    cdef cnp.ndarray[DTYPE_t, ndim=3] out
    
    n = tc.shape[1]
    out = np.empty((5, n, len(lat)), dtype=DTYPE)
    for i in range(n):
        v       = tc[APARENT_SIDEREAL_TIME, i]                                  # ν
        xi      = tc[EQUATOIRAL_HORIZONAL_PARALAX, i]                           # ξ
        delta   = tc[TOPOCENTRIC_DECLINATION, i]                                # δ
        alpha   = tc[TOPOCENTRIC_RIGHT_ASCENSION, i]                            # α

        # - 3.11. Calculate the observer local hour angle, H (in degrees):
        H = (v + lon - alpha) % 360
        
        (
            delta_alpha, delta_p
        ) = topocentric_parallax_right_ascension_and_declination(delta, H, E, lat, xi)

        H_p = H - delta_alpha                                                   # H' = H − ∆α

        # 3.14.1
        (
            e, e0, theta, theta0                                                               # e, e0
        ) = topocentric_azimuth_angle(lat, delta_p, H_p, P, T,  refct)
        

        gamma = topocentric_astronomers_azimuth(H_p, delta_p, lat)
        
        
        
        out[0, i, :] = 90 - e#np.asarray(90 - e).flatten()                                                 # theta[apparent zenith angle]
        out[1, i, :] = 90 - e0                                                # theta0[zenith angle]
        out[2, i, :] = e                                                      # epsilon[elevation angle]
        out[3, i, :] = e0                                                     # epsilon0[apparent elevation angle]
        out[4, i, :] = (gamma + 180) % 360                                    # gamma[azimuth angle]





    return out

def main(
    datetime_like: ArrayLike,
    latitude: ArrayLike,
    longitude: ArrayLike,
    elevation: ArrayLike | None = None,
    pressure: ArrayLike | None = None,
    temperature: ArrayLike | None = None,
    refraction: ArrayLike | None = None,
    apply_correction = False,
    int num_threads = 1,
):
    
    latitude = np.asfarray(latitude, dtype=np.float64)
    longitude = np.asfarray(longitude, dtype=np.float64)
    time_components = fastspa.time_components(datetime_like)
    return c_main(
        time_components,
        latitude,
        longitude,
        np.full_like(latitude, 0.0),
        1013.25,
        12.0,
        0.0,
        # np.full_like(latitude, 1013.25),
        # np.full_like(latitude, 12.0),
        # np.full_like(latitude, 0.0),
        num_threads
    )



import fastspa
import pvlib.spa
time = ["2022-01-01"]
lats = [45.0, 45.0, -45.0, -45.0, 0.0]
lons = [0.0, 90.0, 0.0, 90.0, 0.0]
lats, lons = np.meshgrid(lats, lons)
lats = lats.flatten()
lons = lons.flatten()
dt = np.array(["2022-01-01"]).astype("datetime64[ns]")
ut = dt.astype("int64") / 1e9
delta_t = fastspa.pe4dt(dt)
elevation = np.zeros_like(lats)
import timeit
print(
    timeit.timeit(lambda: np.asarray(main(dt, lats, lons, elevation))),
    timeit.timeit(lambda: fastspa.fast_spa(dt,lats,lons).flatten()),
#     pvlib.spa.solar_position(   
#         ut,
#         [45.0],
#         [0.0],
#         0.0,
#         1013.25,
#         temp=12,
#         delta_t=delta_t,
#         atmos_refract=0,
# ).flatten()[:-1],
    # [158.0107302 , 158.0107302 , -68.0107302 , -68.0107302 ,357.97091567],
    sep='\n'
)
# array([158.0107302 , 158.0107302 , -68.0107302 , -68.0107302 ,357.97091567])

In [9]:
import pvlib.spa
import fastspa._core as fastspa
import numpy as np
print(len(time))





def slow_spa(
    obj,
    lat,
    lon,
    elevation,
    pressure,
    temp,
    refraction,
):
    delta_t = fastspa.pe4dt(obj)
    dt = np.asanyarray(obj, dtype="datetime64[ns]")
    unix_time = dt.astype(np.float64) // 1e9

    x = np.stack(
        [
            np.stack(
                pvlib.spa.solar_position_numpy(
                    ut,
                    lat=lat,
                    lon=lon,
                    elev=elevation,
                    pressure=pressure,
                    temp=temp,
                    delta_t=delta_t[i : i + 1],
                    atmos_refract=refraction,
                    numthreads=None,
                    sst=False,
                    esd=False,
                )[:-1]
            )
            for i, ut in enumerate(unix_time)
        ],
        axis=1,
    )
    return x


dt = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[h]').astype('datetime64[m]')
lats = np.linspace(25, 50, 10)
lons = np.linspace(-125, -65, 10)
lats, lons = np.meshgrid(lats, lons)

%timeit slow_spa(dt, lats, lons, 0.0, 1013.25, temp=12,refraction=0)
%timeit fastspa.main(dt, lats, lons)

175320
10 s ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
426 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import pvlib.spa
import fastspa._core as fastspa
import numpy as np
time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
# lats = [45.0, 45.0, -45.0, -45.0, 0.0]
# lons = [0.0, 90.0, 0.0, 90.0, 0.0]
lats = np.linspace(25, 50, 100)
lons = np.linspace(-125, -65, 100)
lats, lons = np.meshgrid(lats, lons)
# lon, lat = np.meshgrid(lons, lats)




%timeit slow_spa(time, lats, lons, 0.0, 1013.25, 12, 0)
%timeit fastspa.main(time, lats, lons, num_threads=12)
# ) #pvlib.spa.solar_position(time, lats, lons, 0.0, 1013.25, temp=12, delta_t=delta_t, atmos_refract=0)


In [ ]:
fs = fastspa.main(time, lats, lons, num_threads=12)
fs.shape
ss = slow_spa(time, lats, lons, 0.0, 1013.25, 12, 0)
assert fs.shape == ss.shape
print(fs.shape, ss.shape)
fs[0,0], ss[0,0]
# np.allclose(fs, ss, atol=1)

In [ ]:

import numpy as np
import fastspa._core as fastspa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes



lats = np.linspace(25, 50, 100)
lons = np.linspace(-125, -65, 100)
lats, lons = np.meshgrid(lats, lons)

time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
data = fastspa.main(time, lats, lons)

fig, axes = plt.subplots(
    nrows=len(time),
    ncols=data.shape[0],
    figsize=(12, 15),
    subplot_kw=dict(projection=ccrs.PlateCarree()),
)

fig.tight_layout()

for i in range(len(time)):
    for j in range(data.shape[0]):
        t = time[i]
        ax = axes[i, j]
        assert isinstance(ax, GeoAxes)
        ax.coastlines()
        ax.contourf(lons, lats, data[j, i, :, :], transform=ccrs.PlateCarree())
        ax.set_title(f'{t}')




In [ ]:
lats = np.linspace(25, 50, 100)
lons = np.linspace(-125, -65, 100)
lats, lons = np.meshgrid(lats, lons)

time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
data = slow_spa(time, lats, lons, 0.0, 1013.25, 12, 0)

fig, axes = plt.subplots(
    nrows=len(time),
    ncols=data.shape[0],
    figsize=(12, 15),
    subplot_kw=dict(projection=ccrs.PlateCarree()),
)

fig.tight_layout()

for i in range(len(time)):
    for j in range(data.shape[0]):
        t = time[i]
        ax = axes[i, j]
        assert isinstance(ax, GeoAxes)
        ax.coastlines()
        ax.contourf(lons, lats, data[j, i, :, :], transform=ccrs.PlateCarree())
        ax.set_title(f'{t}')



In [ ]:

# import numpy as np
# import fastspa
# lats = np.linspace(25, 50, 100)
# lons = np.linspace(-125, -65, 100)
# lats, lons = np.meshgrid(lats, lons)
# elevation = np.zeros_like(lats)
# temp = np.zeros_like(lats) + 12
# pressure = np.zeros_like(lats) + 1013.25
# refraction = np.zeros_like(lats) + 0.5667

# time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
# %timeit fastspa.fast_spa(time, lats, lons, elevation, pressure, temp, refraction, num_threads=12)

In [ ]:
import numpy as np
import fastspa.main
lats = np.linspace(25, 50, 100)#[:, np.newaxis]
lons = np.linspace(-125, -65, 100)#[:, np.newaxis]
# lats, lons = np.meshgrid(lats, lons)
lats = lats.ravel()[:, np.newaxis]
lons = lons.ravel()[:, np.newaxis]
elevation = np.zeros_like(lats)
temp = np.zeros_like(lats) + 12
pressure = np.zeros_like(lats) + 1013.25
refraction = np.zeros_like(lats) + 0.5667

time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
%timeit np.stack(fastspa.main.solar_position(time, lats, lons, elevation, pressure, temp, refraction, num_threads=1))

In [ ]:
%timeit fastspa.fast_spa(time, lats, lons, elevation, pressure, temp, refraction, num_threads=12)

In [ ]:
import pvlib.spa
# lats = np.linspace(25, 50, 100)
# lons = np.linspace(-125, -65, 100)
# lats, lons = np.meshgrid(lats, lons)

time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')

def slow_spa(
    obj,
    lat,
    lon,
    elevation=0,
    pressure=1013.25,
    temp=12,
    refraction=0.5667,
):
    delta_t = fastspa.pe4dt(obj)
    dt = np.asanyarray(obj, dtype="datetime64[ns]")
    unix_time = dt.astype(np.float64) // 1e9

    x = np.stack(
        [
            np.stack(
                pvlib.spa.solar_position_numpy(
                    ut,
                    lat=lat,
                    lon=lon,
                    elev=elevation,
                    pressure=pressure,
                    temp=temp,
                    delta_t=delta_t[i : i + 1],
                    atmos_refract=refraction,
                    numthreads=None,
                    sst=False,
                    esd=False,
                )[:-1]
            )
            for i, ut in enumerate(unix_time)
        ],
        axis=1,
    )
    return x


%timeit slow_spa(time, lats, lons, elevation, pressure, temp, refraction)

In [ ]:
%%cython
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
cimport cython
from cython.parallel cimport prange
import numpy as np
cimport numpy as cnp

cnp.import_array()

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double[:,:,:] main(int i, int j, int k, int num_threads=1):
    cdef double[:, :, : ] x 
    cdef int i_, j_, k_
    x = np.zeros((i, j, k))
    for i_ in prange(i, nogil=True, num_threads=num_threads):
        for j_ in range(j):
            for k_ in range(k):
            # for k_ in prange(k, nogil=True):
                x[i_, j_, k_] = i_ + j_ + k_
    
    return x
import timeit    
print(timeit.timeit(lambda:np.asfarray(main(5, 5, 5))))
# print(np.asfarray(main(5, 5, 5)))



In [ ]:
import glob
# find all pyx and pxd files

def extension_modules(src:str, x:str):
    patterns = glob.glob(x, root_dir=src)
    names = [f"{src}.{p.split('.')[0]}" for p in patterns]
    paths = [[f"{src}/{p}"] for p in patterns]
    return [
        {
            "name": name,
            "sources": path,
        } for name, path in zip(names, paths)
    ]
extension_modules('fastspa', "*[.pyx][.pxd]")